# Assignment 8 - [#1079](https://github.com/alexanderquispe/Diplomado_PUCP/issues/1079)
**Deadline: September 14th - 23:59**

Fernando Mendoza | Andrea Pezo | Michel Cotrina | Estefanny Gil | Armando Ore

## Instructions

1. Get the 15 variables from this raster for all Peru departments polygons. This is the [link](https://github.com/alexanderquispe/Diplomado_PUCP/tree/main/_data/INEI_LIMITE_DEPARTAMENTAL) where shapefiles are located. This is the [link](https://ghsl.jrc.ec.europa.eu/download.php?ds=builtC) of the source raster. The values should be the percentage of district area cover by this specific Morphological Settlement Zone.
2. Then you are going to generate choropleth map using folium for these 15 variables.
3. Save your html in the same folder of your JN. Name your HTML as your branch. This HTML should have all these layers. Please do not forget to use Layer Control.

**Do not upload any other file. Be cautious of uploading files greater than 100MB.**

In [23]:
pwd

'C:\\Users\\LENOVO\\Desktop\\Github\\Diplomado_PUCP\\Lecture_11\\Assignment_8'

## Step 0: Library

## Step 1: 